In [2]:
!ls

installing  scpt-private-ethereum-network  Untitled.ipynb  workshop


In [5]:
# workshop/safe-smart-contracts-workshop-08-2015
!cat workshop/safe-smart-contracts-workshop-08-2015/example-01-basics.py

from ethereum import tester

# Logging (change to "trace" to see more)
#from ethereum import slogging
#slogging.set_level('eth.pb.tx', 'warning')
#slogging.set_level('eth.vm.op', 'warning')

# Serpent code
contract_code = """
event Notice(x:str)

def test_function(x):
   log(type=Notice, text("Hello World!"))

   self.storage[0] = x
   self.storage[0xDEAD] = 0xBEEF
   self.storage["Hello"] = "World!"
   self.storage[1] = 10 + 1245 + 320   # 1575 = 0x0627

"""

s = tester.state()

# Create the contract
full_code = contract_code
contract = s.abi_contract(full_code)

# Invoke a method (in the local "mempool", to go in the next block)
contract.test_function("C0FFEEBABE".decode('hex'))

# Mine blocks
s.mine(10)

# Inspect the contract storage
def simplify_trie(trie):
    return dict((k.encode('hex'), v[1:].encode('hex')) 
                for k,v in trie.to_dict().iteritems())

for k,v in simplify_trie(s.block.get_storage(contract.address)).iteritems():
    print k,v

# Print the balance of 

In [4]:
run workshop/safe-smart-contracts-workshop-08-2015/example-01-basics.py

{u'x': 'Hello World!', '_event_type': 'Notice'}
0000000000000000000000000000000000000000000000000000000000000000 c0ffeebabe
0000000000000000000000000000000000000000000000000000000000000001 0627
000000000000000000000000000000000000000000000000000000000000dead beef
48656c6c6f000000000000000000000000000000000000000000000000000000 576f726c64210000000000000000000000000000000000000000000000000000


/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/ubuntu/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)
No handlers could be found for logger "eth.pow"


In [6]:
from ethereum import tester

# Logging (change to "trace" to see more)
#from ethereum import slogging
#slogging.set_level('eth.pb.tx', 'warning')
#slogging.set_level('eth.vm.op', 'warning')

# Serpent code
contract_code = """
event Notice(x:str)

def test_function(x):
   log(type=Notice, text("Hello World!"))

   self.storage[0] = x
   self.storage[0xDEAD] = 0xBEEF
   self.storage["Hello"] = "World!"
   self.storage[1] = 10 + 1245 + 320   # 1575 = 0x0627

"""

s = tester.state()

# Create the contract
full_code = contract_code
contract = s.abi_contract(full_code)

# Invoke a method (in the local "mempool", to go in the next block)
contract.test_function("C0FFEEBABE".decode('hex'))

# Mine blocks
s.mine(10)

# Inspect the contract storage
def simplify_trie(trie):
    return dict((k.encode('hex'), v[1:].encode('hex')) 
                for k,v in trie.to_dict().iteritems())

for k,v in simplify_trie(s.block.get_storage(contract.address)).iteritems():
    print k,v

# Print the balance of Alice
#print "[1] Alice' balance", s.block.get_balance(alice)
#s.send(tester.k0, contract.address, 1)
#print "[2] Alice' balance", s.block.get_balance(alice)


# Look at compiled code
# print serpent.pretty_compile(contract_code)

# Look at intermediate LLL language
# print serpent.pretty_compile(contract_code)


{u'x': 'Hello World!', '_event_type': 'Notice'}
0000000000000000000000000000000000000000000000000000000000000000 c0ffeebabe
0000000000000000000000000000000000000000000000000000000000000001 0627
000000000000000000000000000000000000000000000000000000000000dead beef
48656c6c6f000000000000000000000000000000000000000000000000000000 576f726c64210000000000000000000000000000000000000000000000000000


In [8]:
tester.a0

'\x82\xa9x\xb3\xf5\x96*[\tW\xd9\xee\x9e\xefG.\xe5[B\xf1'

In [10]:
tester.a8

'\xe0\xfc\x04\xfa-4\xa6kw\x9f\xd5\xce\xe7H&\x802\xa1F\xc0'

In [11]:
!cat workshop/safe-smart-contracts-workshop-08-2015/example-02-buggy-lottery.py

from ethereum import tester
import os

contract_code = """
event Notice(x:str)

alice = 0x{alice}
bob = 0x{bob}

def load_money():
   # Must pay before time #1
   if block.number > 1: 
      log(type=Notice, text("load_money: called after block 1"))
      return(0)

   if msg.value != 10*(10**21):
      log(type=Notice, text("load_money: wrong amount"))
      return(0)

   if msg.sender == alice:
      log(type=Notice, text("load_money: Alice OK!"))

   if msg.sender == bob:
      log(type=Notice, text("load_money: Bob OK!"))

   # Mark user as having paid
   self.storage[msg.sender] = 1 

def cash_out():
   # Critical block is #2
   # Must wait for block #3 to cash out
   if block.number < 3:
      log(type=Notice, text("cash_out: called before block 3"))
      return(0)

   if not self.storage[alice]:
      log(type=Notice, text("cash_out: Alice didn't pay!"))
      send(bob, self.balance)
      return(0)

   if not self.storage[bob]:
      log(type=Notice, text("cash_out: Bob didn't